In [1]:
# Importing needed library
import pandas as pd
import numpy as np
import beta_glu
import pandascharm as pc
from sklearn.metrics import r2_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

from sklearn.preprocessing import OneHotEncoder
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
import blosum as bl
from Bio.SubsMat.MatrixInfo import blosum62 as blosum
from Bio import AlignIO
from Bio import SeqIO
from Bio.Align.Applications import MuscleCommandline
from Bio.Align import AlignInfo
from scipy.fft import fft, ifft

from sklearn.preprocessing import scale 
from sklearn import model_selection
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import train_test_split
from sklearn.cross_decomposition import PLSRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import r2_score
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from pyaaisc import Aaindex
from Bio import AlignIO


/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/Bio/SubsMat/__init__.py:126: BiopythonDeprecationWarning: Bio.SubsMat has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.substitution_matrices as a replacement, and contact the Biopython developers if you still need the Bio.SubsMat module.
  warnings.warn(


In [2]:
df = pd.read_excel('../beta-glucosidase/data/KcatKm w_ Temperatures.xlsx')

In [3]:
output = 'pNP-Glc kcat/Km (1/smM)'
encoding = 'BLOSUM45'
aln = '../beta-glucosidase/data/kcatKm w_Temperatures.aln'

In [4]:
X,y,holder = beta_glu.encode_temp(encoding, output, df, aln, key = None)

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=101)

In [6]:
all_result = []

In [7]:
# DT Tuning
def DT(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid={ "max_features": [2,4,6,8,10,12], 'random_state':[20]}
    tree_reg=DecisionTreeRegressor()
    if jack == False:
        grid_search=GridSearchCV(tree_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(tree_reg, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    DT_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return DT_pred, pred_training, param

In [8]:
DT_pred , DT_train, param = DT(X_train, y_train, X_val)

In [9]:
all_result.append([output, "Decision Tree Regression", encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, DT_train)), np.sqrt(metrics.mean_squared_error(y_val, DT_pred)), r2_score(y_train, DT_train),r2_score(y_val, DT_pred), str(param)]) 

In [10]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'BLOSUM45',
  nan,
  5.428134099159952,
  116.95244593479505,
  0.9991613857443024,
  0.6287021951680001,
  "{'max_features': 12, 'random_state': 20}"]]

In [11]:
# SVM Tuning
def SVM(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid = {'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5]}
 
    model = SVR()
    if jack == False:
        grid_search=GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(model, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    SVM_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return SVM_pred, pred_training, param

In [12]:
SVM_pred , SVM_train, param = SVM(X_train, y_train, X_val)

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when 

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when 

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when 

In [13]:
all_result.append([output, "Support Vector Machine Regression", encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, SVM_train)), np.sqrt(metrics.mean_squared_error(y_val, SVM_pred)), r2_score(y_train, SVM_train),r2_score(y_val, SVM_pred), param])

In [14]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'BLOSUM45',
  nan,
  5.428134099159952,
  116.95244593479505,
  0.9991613857443024,
  0.6287021951680001,
  "{'max_features': 12, 'random_state': 20}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Support Vector Machine Regression',
  'BLOSUM45',
  nan,
  184.29327078512998,
  194.07844046305038,
  0.03332595023844742,
  -0.022488308841060567,
  {'C': 10, 'coef0': 0.01, 'degree': 3}]]

In [15]:
param

{'C': 10, 'coef0': 0.01, 'degree': 3}

In [16]:
# Random Forest Tuning
def RF(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 50, stop = 150, num = 10)]
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Create the random grid
    param_grid = {'n_estimators': n_estimators,
                   'max_depth': max_depth,
                   'min_samples_split': min_samples_split,
                   'min_samples_leaf': min_samples_leaf}
    rf = RandomForestRegressor()
    if jack == False:
        grid_search=GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(rf, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train,np.ravel(y_train))
    RF_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return RF_pred, pred_training, param

In [17]:
RF_pred , RF_train, param = RF(X_train, y_train, X_val)

In [18]:
all_result.append([output, "Random Forest Regression", encoding,holder,np.sqrt(metrics.mean_squared_error(y_train, RF_train)), np.sqrt(metrics.mean_squared_error(y_val, RF_pred)), r2_score(y_train, RF_train),r2_score(y_val, RF_pred),str(param)])

In [19]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'BLOSUM45',
  nan,
  5.428134099159952,
  116.95244593479505,
  0.9991613857443024,
  0.6287021951680001,
  "{'max_features': 12, 'random_state': 20}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Support Vector Machine Regression',
  'BLOSUM45',
  nan,
  184.29327078512998,
  194.07844046305038,
  0.03332595023844742,
  -0.022488308841060567,
  {'C': 10, 'coef0': 0.01, 'degree': 3}],
 ['pNP-Glc kcat/Km (1/smM)',
  'Random Forest Regression',
  'BLOSUM45',
  nan,
  42.61369754391727,
  118.40459100780087,
  0.9483155924170076,
  0.6194244816716752,
  "{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 61}"]]

In [20]:
param

{'max_depth': 10,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 61}

In [21]:
# XGBR tuning
def XGBR(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    
    param_grid = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5]
        }
    
    if jack == False:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(xgb.XGBRegressor(), param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    XGB_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return XGB_pred, pred_training, param


In [22]:
XG_pred, XG_train, param = XGBR(X_train, y_train, X_val)

In [23]:
all_result.append([output,"XGB Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, XG_train)), np.sqrt(metrics.mean_squared_error(y_val, XG_pred)), r2_score(y_train, XG_train),r2_score(y_val, XG_pred),str(param)]) 

In [24]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'BLOSUM45',
  nan,
  5.428134099159952,
  116.95244593479505,
  0.9991613857443024,
  0.6287021951680001,
  "{'max_features': 12, 'random_state': 20}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Support Vector Machine Regression',
  'BLOSUM45',
  nan,
  184.29327078512998,
  194.07844046305038,
  0.03332595023844742,
  -0.022488308841060567,
  {'C': 10, 'coef0': 0.01, 'degree': 3}],
 ['pNP-Glc kcat/Km (1/smM)',
  'Random Forest Regression',
  'BLOSUM45',
  nan,
  42.61369754391727,
  118.40459100780087,
  0.9483155924170076,
  0.6194244816716752,
  "{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 61}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'XGB Regression',
  'BLOSUM45',
  nan,
  15.864110363416755,
  107.29616741303212,
  0.9928370396115016,
  0.6874840537577462,
  "{'colsample_bytree': 0.6, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 1.0}"]]

In [25]:
param

{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'max_depth': 5,
 'min_child_weight': 1,
 'subsample': 1.0}

In [26]:
# NNR Tuning
def NNR(X_train, y_train, X_val, jack = False):
    
    knife = len(X_train)-1
    param_grid = {"hidden_layer_sizes": [1,50], "activation": ["identity", "logistic", "tanh", "relu"], "solver": ["lbfgs", "sgd", "adam"], "alpha": [0.00005,0.0005]}
    regr = MLPRegressor(random_state=101, max_iter=100).fit(X_train, y_train)
    if jack == False:
        grid_search=GridSearchCV(regr, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(regr, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    NNR_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return NNR_pred, pred_training, param

In [27]:
NNR_pred, NNR_train, param = NNR(X_train, y_train, X_val)

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

...

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_regression.py", line 438, in mean_squared_error
    y_type, y_true, y_pred, multioutput = _check_reg_targets(
  File "/Users/ryanjang123/opt/an

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1599: DataConversionWarning: A column-vector y was passed when a 1d array was expecte

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_p

In [28]:
all_result.append([output,"Neural Network Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, NNR_train)), np.sqrt(metrics.mean_squared_error(y_val, NNR_pred)), r2_score(y_train, NNR_train),r2_score(y_val, NNR_pred),str(param)]) 

In [29]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'BLOSUM45',
  nan,
  5.428134099159952,
  116.95244593479505,
  0.9991613857443024,
  0.6287021951680001,
  "{'max_features': 12, 'random_state': 20}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Support Vector Machine Regression',
  'BLOSUM45',
  nan,
  184.29327078512998,
  194.07844046305038,
  0.03332595023844742,
  -0.022488308841060567,
  {'C': 10, 'coef0': 0.01, 'degree': 3}],
 ['pNP-Glc kcat/Km (1/smM)',
  'Random Forest Regression',
  'BLOSUM45',
  nan,
  42.61369754391727,
  118.40459100780087,
  0.9483155924170076,
  0.6194244816716752,
  "{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 61}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'XGB Regression',
  'BLOSUM45',
  nan,
  15.864110363416755,
  107.29616741303212,
  0.9928370396115016,
  0.6874840537577462,
  "{'colsample_bytree': 0.6, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 1.0}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Neural Netwo

In [30]:
#Elastic Net

def EN(X_train, y_train, X_val, jack = False):
    knife = len(X_train)-1
    param_grid = {"max_iter": [1, 5, 10], "alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],"l1_ratio": np.arange(0.0, 1.0, 0.1)}
    ENmodel = ElasticNet()
    if jack == False:
        grid_search=GridSearchCV(ENmodel, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
    else:
        grid_search=GridSearchCV(ENmodel, param_grid, cv=knife, scoring='neg_mean_squared_error', return_train_score=True)
    grid_search.fit(X_train, y_train)
    EN_pred = grid_search.predict(X_val)
    pred_training= grid_search.predict(X_train)
    param = grid_search.best_params_
    return EN_pred, pred_training,param

In [31]:
EN_pred , EN_train, param = EN(X_train, y_train, X_val)

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.661e+06, tolerance: 1.023e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.162e+06, tolerance: 1.149e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the 

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.114e+06, tolerance: 1.082e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.661e+06, tolerance: 1.023e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.858e+06, tolerance: 1.149e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.442e+06, tolerance: 1.144e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.080e+06, tolerance: 1.023e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.346e+06, tolerance: 1.149e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.701e+06, tolerance: 1.144e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.324e+06, tolerance: 1.236e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the 

...

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.126e+06, tolerance: 1.023e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.753e+06, tolerance: 1.149e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.067e+06, tolerance: 1.144e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.215e+06, tolerance: 1.236e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.073e+06, tolerance: 1.144e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.216e+06, tolerance: 1.236e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.027e+06, tolerance: 1.082e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.834e+06, tolerance: 1.023e+03 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the 

/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.297e+03, tolerance: 1.149e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.420e+03, tolerance: 1.144e+03
  model = cd_fast.enet_coordinate_descent(
/Users/ryanjang123/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider incre

In [32]:
all_result.append([output,"Elastic Net Regression",encoding,holder, np.sqrt(metrics.mean_squared_error(y_train, EN_train)), np.sqrt(metrics.mean_squared_error(y_val, EN_pred)), r2_score(y_train, EN_train),r2_score(y_val, EN_pred),str(param)])

In [33]:
all_result

[['pNP-Glc kcat/Km (1/smM)',
  'Decision Tree Regression',
  'BLOSUM45',
  nan,
  5.428134099159952,
  116.95244593479505,
  0.9991613857443024,
  0.6287021951680001,
  "{'max_features': 12, 'random_state': 20}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Support Vector Machine Regression',
  'BLOSUM45',
  nan,
  184.29327078512998,
  194.07844046305038,
  0.03332595023844742,
  -0.022488308841060567,
  {'C': 10, 'coef0': 0.01, 'degree': 3}],
 ['pNP-Glc kcat/Km (1/smM)',
  'Random Forest Regression',
  'BLOSUM45',
  nan,
  42.61369754391727,
  118.40459100780087,
  0.9483155924170076,
  0.6194244816716752,
  "{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 61}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'XGB Regression',
  'BLOSUM45',
  nan,
  15.864110363416755,
  107.29616741303212,
  0.9928370396115016,
  0.6874840537577462,
  "{'colsample_bytree': 0.6, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 1.0}"],
 ['pNP-Glc kcat/Km (1/smM)',
  'Neural Netwo

In [34]:
dfResults = pd.DataFrame(all_result, columns=['Output' , 'Algorithm','Encoding Method' ,'Code', "RMSE Training", 'RMSE Val',"R^2 train","R^2 Val","Best_Parameter"])

In [35]:
dfResults

,Output,Algorithm,Encoding Method,Code,RMSE Training,RMSE Val,R^2 train,R^2 Val,Best_Parameter
0,pNP-Glc kcat/Km (1/smM),Decision Tree Regression,BLOSUM45,NaN,5.428134,116.952446,0.999161,0.628702,"{'max_features': 12, 'random_state': 20}"
1,pNP-Glc kcat/Km (1/smM),Support Vector Machine Regression,BLOSUM45,NaN,184.293271,194.078440,0.033326,-0.022488,"{'C': 10, 'coef0': 0.01, 'degree': 3}"
2,pNP-Glc kcat/Km (1/smM),Random Forest Regression,BLOSUM45,NaN,42.613698,118.404591,0.948316,0.619424,"{'max_depth': 10, 'min_samples_leaf': 1, 'min_..."
3,pNP-Glc kcat/Km (1/smM),XGB Regression,BLOSUM45,NaN,15.864110,107.296167,0.992837,0.687484,"{'colsample_bytree': 0.6, 'gamma': 0.5, 'max_d..."
4,pNP-Glc kcat/Km (1/smM),Neural Network Regression,BLOSUM45,NaN,103.970203,140.079466,0.692334,0.467337,"{'activation': 'relu', 'alpha': 0.0005, 'hidde..."
5,pNP-Glc kcat/Km (1/smM),Elastic Net Regression,BLOSUM45,NaN,123.853027,147.608521,0.563409,0.408538,"{'alpha': 1, 'l1_ratio': 0.30000000000000004, ..."


In [36]:
name = encoding + "_hyperparameter_tune_new.xlsx"
dfResults.to_excel(name)  